# Readout modeling
## Full sophistication



In [8]:
# import numpy as np
import scipy.signal as signal
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(110)
import seaborn as sns

sns.set_style("white")
matplotlib.rcParams['figure.figsize'] = [20, 15]

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from ipywidgets import interact, interactive, fixed


def try_coverage(coverage=1.0,injection=0.0, noise_level=0.0):
    behav_dim = 30
    brain_dim = 50
    measure_dim = 2

    gamma_ratio = 1/50
    gamma_b = np.random.choice([0,1],size=(behav_dim,brain_dim),p=[1-gamma_ratio,gamma_ratio])
    #Here we want to determine h_b parametrized by a coverage parameter [0,1]
    disease_regions = np.sum(gamma_b,axis=1) #behav x 1
    brain_importance = np.sum(gamma_b,axis=0) # bran x 1
    
    #find all non-zero brain regions
    b_i_idx = np.where(brain_importance > 0)[0]
    #now take only coverage * b_i_idx regions randomly and that's your h
    measurables = np.random.choice(b_i_idx,replace=False,size=(np.ceil(len(b_i_idx) * coverage).astype(np.int),))
    #print(brain_importance)
    #print(measurables)

    h_b = np.zeros(shape=(len(measurables),brain_dim))
    for ii,br in enumerate(measurables):
        h_b[ii][br] = 1

    injection = 0
    #now do injection (including non-disease related brain regions into measurements)
    b_n_idx = np.where(brain_importance == 0)[0] #brain, NOT important
    #print(b_n_idx)
    noise_sources = np.random.choice(b_n_idx,replace=False,size=(np.ceil(len(b_n_idx) * injection).astype(np.int),))
    for ii,br in enumerate(noise_sources):
        h_b[ii][br] = 1
    
    #h_b = gamma_b
    #h_b = np.random.choice([0,1],size=(measure_dim,brain_dim),p=[2/3,1/3])

    N = 200
    x = np.random.normal(0,1,size=(brain_dim,N))

    beta = np.dot(gamma_b,x)
    c = np.sum(beta,axis=0)
    y = (np.dot(h_b,x))
    y += np.random.normal(0,noise_level,size=y.shape)

    readout_model = LinearRegression().fit(y.T,c)

    Nt = 1000
    x_test = np.random.normal(0,1,size=(brain_dim,Nt))
    beta_test = np.dot(gamma_b,x_test)
    c_test = np.sum(beta_test,axis=0)
    y_test = np.dot(h_b,x_test)

    pred_c = readout_model.predict(y_test.T)

    plt.subplot(221)
    plt.scatter(pred_c,c_test)
    plt.xlabel('Predicted');plt.ylabel('Actual')
    assess_lr = LinearRegression(fit_intercept=True).fit(pred_c.reshape(-1,1),c_test.reshape(-1,1))
    r2_fit_pva = r2_score(c_test,pred_c) #have to be careful here because the function takes (actual, then predicted) in arguments
    plt.title('Slope'+str(assess_lr.coef_))
    plt.plot([-10,10],[-10,10],linestyle='dotted')

    plt.subplot(222)
    plt.scatter(c_test,pred_c)
    plt.xlabel('Actual');plt.ylabel('Predicted')
    assess_rl = LinearRegression(fit_intercept=True).fit(c_test.reshape(-1,1),pred_c.reshape(-1,1))
    r2_fit_avp = r2_score(pred_c,c_test)
    plt.title('Slope'+str(assess_rl.coef_))
    plt.plot([-10,10],[-10,10],linestyle='dotted')
    
    plt.subplot(2,1,2)
    plt.imshow(h_b)
    
    plt.suptitle('R2:' + str((r2_fit_pva,'vs',r2_fit_avp)))
    
readout_widg = interactive(try_coverage,coverage=(0.0,1.0,0.01),noise_level=(0.0,10.0,0.1))
display(readout_widg)

interactive(children=(FloatSlider(value=1.0, description='coverage', max=1.0, step=0.01), FloatSlider(value=0.…

In the above example **coverage** refers to what percentage of the number of brain regions known to be involved in behavior are actually being recorded from.

**Injection** refers to how many brain regions known to *not* be involved in the disease mapping are being recorded from.

Noise is the additive noise component in the final measurement of $\vec{y}$.

## Working through the math
Let's work through the math behind the difference a bit.

### Actual-on-Y
Putting the 'Actual' Value on the y-axis is convention in machine learning approaches.
This is largely because the goal is to assess how something we have access to and can calculate (predicted) maps to the single 

When we do 'Actual vs Predicted' (In the Y-vs-X convention) we're finding $m$:

$$ \beta = m \cdot \hat{\beta}$$

If $\hat{\beta}$ has any internal component that is correlated with $\beta$ then it's trivial to find a scaling within the calculation of $\hat{\beta}$ that can admit $m = 1$.
So the slope doesn't really mean much.

You have no control over A

### Actual-on-X

$m$ corresponds to the slope . In the probabilistic framework:

$$ \beta \sim \mathcal{N}(m\cdot\hat{\beta},\sigma^2)$$